In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner
import hazm
from collections import defaultdict




In [2]:
data = pd.read_csv('./culture.csv')

In [3]:
def delete_stop_words(word):    
    if len(word)<3:
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

In [4]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

In [5]:
text = data['mText'][6]
all_words = nltk.word_tokenize(unicode(text,'utf-8'))
words = clean_text(text)

In [6]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [7]:
position_dict= defaultdict(list)
for i in words:
    position_dict[i['Word']].append(i['Position'])
    
for i in position_dict:
    position_dict[i] = sorted(position_dict[i])

In [8]:
life_time = defaultdict(int)
for i in position_dict:
    n = position_dict[i][-1]-position_dict[i][0] -1 
    if n == -1 :
        n = 0
    life_time[i] = float(n)/len(all_words)

In [9]:
words_dict = []
for i in position_dict:
    n = defaultdict()
    n['Word'] = i
    n['Position'] = position_dict[i]
    n['life_time'] = life_time[i]
    n['tf'] = len(position_dict[i])
    words_dict.append(n)

In [21]:
mean = float(sum([i['tf'] for i in words_dict]))/len(words_dict)
for word in words_dict:

    word['Var'] = word['tf'] - mean

In [28]:
for word in words_dict:
    word['Weight'] = (((1-0.7)*word['life_time']) + 0.7*(word['tf']+word['Var'])) - word['Var']

    

In [29]:
sorted_list = sorted(words_dict,key = lambda x : x['Weight'],reverse=True)

In [30]:

for i in sorted_list:
    print(i['Word'])
    print('Weight: ',i['Weight'])
    print('var: ',i['Var'])
    print('life time: ',i['life_time'])
    print('Repitition: ',i['tf'])

معروف
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
میراث
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
بدنه
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
پیام
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
بانی
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
زنده
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
حضور
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
انجام
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
امسال
('Weight: ', 1.902659574468085)
('var: ', -0.7074468085106382)
('life time: ', 0.0)
('Repitition: ', 1)
افزود#افزا
('We

In [27]:
n = 0 
for i in words_dict:
    if n

KeyError: 'weight'